In [0]:
! pip install spacy
! python -m spacy download en
! pip install torchtext==0.2.3
!pip install keras

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

In [2]:
from pathlib import Path
# from fastai.text import *
# import preprocessor as pre
# pre.set_options(pre.OPT.URL,pre.OPT.NUMBER,pre.OPT.MENTION,pre.OPT.HASHTAG)

In [3]:
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline,linear_model
from sklearn.metrics import confusion_matrix
# from fastai.plots import plot_confusion_matrix
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import re
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

In [4]:
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import keras

Using TensorFlow backend.


In [12]:
from nltk import word_tokenize
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')
nltk.download('punkt')

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /Users/apple/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /Users/apple/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
PATH=Path("../")

In [6]:
df=pd.read_csv(PATH/'data/data.csv')
df.head(3)

,title,score,id,body,author,flair,url,comms_num,created,comment,authors,combined_features
0,"Lost my Job, Sick Mother and Paralysed Dad, In...",1042,g014wc,Hi....It's really tough time for everyone. I r...,sanand_satwik,AskIndia,https://www.reddit.com/r/india/comments/g014wc...,132,1.586742e+09,I'm a freelancer. Don't listen to the idiots ...,hashedram diabapp xataari Aashayrao sarcrasti...,"Lost my Job, Sick Mother and Paralysed Dad, In..."
1,Why does the government come with a begging bo...,647,fxofyu,"We have floods, terrorist attacks, famines due...",TWO-WHEELER-MAFIA,AskIndia,https://www.reddit.com/r/india/comments/fxofyu...,205,1.586448e+09,I don't understand why they don't use money f...,Kinky-Monk ak32009 fools_eye None DwncstSheep...,Why does the government come with a begging bo...
2,Mother's condition is going worse due to hepat...,756,g0zlly,"Hi folks, I really appreciate the warm respons...",sanand_satwik,AskIndia,https://www.reddit.com/r/india/comments/g0zlly...,92,1.586871e+09,If anyone knows who is influential on Twitter...,AlternativeDrop6 TheRobotsHaveCome lanky32 pl...,Mother's condition is going worse due to hepat...


In [7]:
df['title'] = df['title'].astype(str)+df['body'].astype(str)+df['comment'].astype(str) 

In [8]:
df=df[['flair','title']]
df["flair"].apply(lambda x:str(x))
# df.dropna()

0       AskIndia
1       AskIndia
2       AskIndia
3       AskIndia
4       AskIndia
          ...   
1795      Sports
1796      Sports
1797      Sports
1798      Sports
1799      Sports
Name: flair, Length: 1800, dtype: object

In [9]:
all_flairs=list(np.unique(df['flair']))
df['flair']=df['flair'].apply(lambda x :all_flairs.index(x))
all_flairs=list(np.unique(df['flair']))

In [10]:
df["title"].apply(lambda x:str(x))

0       Lost my Job, Sick Mother and Paralysed Dad, In...
1       Why does the government come with a begging bo...
2       Mother's condition is going worse due to hepat...
3       Men who are 30+ and have decided not to get ma...
4       r/India: If money is no bar, would you prefer ...
                              ...                        
1795    Indian Women's and Men's Hockey Teams Seal Ber...
1796    'It's humiliating for us': village disowns Dut...
1797    Feroz Shah Kotla Renamed Arun Jaitley Stadium,...
1798    Indian cricket fans are the most irritating ha...
1799    Copy India's ambition to be the best: Ian Chap...
Name: title, Length: 1800, dtype: object

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df['title'],df['flair'],random_state=2020)

In [12]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)

In [13]:
from sklearn.metrics import classification_report

In [14]:
# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
count_vect.fit(list(X_train) + list(X_test))
X_train_count =  count_vect.transform(X_train) 
X_test_count = count_vect.transform(X_test)

tfidf_vect.fit(list(X_train) + list(X_test))
X_train_tfidf =  tfidf_vect.transform(X_train) 
X_test_tfidf = tfidf_vect.transform(X_test)

tfidf_vect_ngram.fit(list(X_train) + list(X_test))
X_train_tfidf_ngram =  tfidf_vect_ngram.transform(X_train) 
X_test_tfidf_ngram = tfidf_vect_ngram.transform(X_test)

tfidf_vect_ngram_chars.fit(list(X_train) + list(X_test))
X_train_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(X_train) 
X_test_tfidf_ngram_chars = tfidf_vect_ngram_chars.transform(X_test)

/Users/apple/Desktop/ml/midas_internship/env/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [15]:

# convert to one_hot
train_y_onehot = keras.utils.to_categorical(y_train, max(y_train)+1)
valid_y_onehot = keras.utils.to_categorical(y_test, max(y_train)+1)


In [17]:
def model_FF(X_train, ytrain, X_test, yvalid, hidden_size, epochs =3):
    # create input layer 
    input_layer = layers.Input((X_train.shape[1], ), sparse=False)
    
    # create hidden layer
    hidden_layer = layers.Dense(hidden_size, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(9, activation="softmax")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    classifier.fit(X_train, ytrain,
                   validation_data=(X_test, yvalid),
                  batch_size=256,
                  epochs=epochs,
                  shuffle = True)
    # scores of the classifier
    predictions = classifier.predict(X_test)
    val_preds = predictions.argmax(axis=-1)
#     accuracy = classifier.evaluate(X_test, yvalid, verbose=0)
    auc = classification_report(y_test, val_preds)
    return auc


# NN Classifier on Count Vectors
accuracy = model_FF(X_train_count, train_y_onehot, X_test_count, valid_y_onehot, hidden_size=100)
print("NN, Count Vectors accuracy:%s"% (accuracy))

# NN Classifier on Word Level TF IDF Vectors
accuracy = model_FF(X_train_tfidf, train_y_onehot, X_test_tfidf, valid_y_onehot, 100)
print("NN, Word Level TF IDF Vectors accuracy:%s "% (accuracy))

# NN Classifier on Ngram Level TF IDF Vectors
accuracy = model_FF(X_train_tfidf_ngram, train_y_onehot, X_test_tfidf_ngram, valid_y_onehot, 100)
print("NN, Ngram Level TF IDF Vectors accuracy:%s"% (accuracy))

# NN Classifier on characters level tf-idf
accuracy = model_FF(X_train_tfidf_ngram_chars, train_y_onehot, X_test_tfidf_ngram_chars, valid_y_onehot, 100)
print("NN, characters level tf-idf Vectors accuracy:%s"% (accuracy))


Train on 1350 samples, validate on 450 samples
Epoch 1/3
1350/1350 [==============================] - 1s 664us/step - loss: 2.0579 - accuracy: 0.2111 - val_loss: 1.8798 - val_accuracy: 0.3844
Epoch 2/3
1350/1350 [==============================] - 1s 435us/step - loss: 1.3630 - accuracy: 0.8222 - val_loss: 1.6557 - val_accuracy: 0.5422
Epoch 3/3
1350/1350 [==============================] - 1s 466us/step - loss: 0.9212 - accuracy: 0.9563 - val_loss: 1.5079 - val_accuracy: 0.6089
NN, Count Vectors accuracy:              precision    recall  f1-score   support

           0       0.50      0.27      0.35        44
           1       0.43      0.61      0.50        44
           2       0.90      0.71      0.79        51
           3       0.37      0.67      0.48        52
           4       0.85      0.80      0.83        51
           5       0.50      0.55      0.52        49
           6       0.64      0.65      0.65        55
           7       0.71      0.43      0.54        51
    

/Users/apple/Desktop/ml/midas_internship/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train on 1350 samples, validate on 450 samples
Epoch 1/3
1350/1350 [==============================] - 0s 144us/step - loss: 2.1898 - accuracy: 0.1207 - val_loss: 2.1730 - val_accuracy: 0.1489
Epoch 2/3
1350/1350 [==============================] - 0s 109us/step - loss: 2.1379 - accuracy: 0.3096 - val_loss: 2.1325 - val_accuracy: 0.3133
Epoch 3/3
1350/1350 [==============================] - 0s 118us/step - loss: 2.0736 - accuracy: 0.4837 - val_loss: 2.0834 - val_accuracy: 0.3889
NN, characters level tf-idf Vectors accuracy:              precision    recall  f1-score   support

           0       0.28      0.84      0.42        44
           1       0.44      0.39      0.41        44
           2       1.00      0.16      0.27        51
           3       0.24      0.63      0.35        52
           4       0.86      0.47      0.61        51
           5       0.33      0.12      0.18        49
           6       0.45      0.55      0.50        55
           7       1.00      0.04      0

In [19]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip -P "../weight_files/"

dyld: Library not loaded: /usr/local/opt/openssl/lib/libssl.1.0.0.dylib
  Referenced from: /usr/local/bin/wget
  Reason: image not found


In [0]:
!unzip '/content/drive/My Drive/models/wiki-news-300d-1M.vec.zip' -d '/content/drive/My Drive/models/'

Archive:  /content/drive/My Drive/models/wiki-news-300d-1M.vec.zip
  inflating: /content/drive/My Drive/models/wiki-news-300d-1M.vec  


In [0]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('/content/drive/My Drive/models/wiki-news-300d-1M.vec', encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(xtrain)
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(xtrain), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(xvalid), maxlen=70)

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        


In [0]:
def cnn(xtrain, ytrain, xvalid, yvalid, eps = 3):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 4, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(2, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer='adam', loss='binary_crossentropy',  metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=eps)
    predictions = model.predict(xvalid)
    val_preds = predictions.argmax(axis=-1)
    accuracy = model.evaluate(xvalid, yvalid, verbose=0)
    f1score=final_score(getf1_score(np.argmax(yvalid,axis=1),val_preds),f1_weight(yvalid,val_preds))
    return accuracy, f1score
  
accuracy, f1score = cnn(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
print("CNN, Word Embeddings acuuracy accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

In [0]:
#lstm unidirectional 128
def lstm(xtrain, ytrain, xvalid, yvalid, epochs = 1):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer1 = layers.LSTM(128)(embedding_layer)
    dropout1 = layers.Dropout(0.5)(lstm_layer1)
    #lstm_layer2 = layers.LSTM(128)(dropout1)
    #dropout2 = layers.Dropout(0.5)(lstm_layer2)
    # Add the output Layers
    output_layer = layers.Dense(2, activation="softmax")(dropout1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=4)
    
    predictions = model.predict(xvalid)
    val_preds = predictions.argmax(axis=-1)
    accuracy = model.evaluate(xvalid, yvalid, verbose=0)
    f1score=final_score(getf1_score(np.argmax(yvalid,axis=1),val_preds),f1_weight(yvalid,val_preds))
    return accuracy, f1score
    
accuracy, f1score = lstm(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
print("LSTM, Word Embeddings accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

Epoch 1/4
8500/8500 [==============================] - 6s 701us/step - loss: 0.5737 - acc: 0.7453
Epoch 2/4
8500/8500 [==============================] - 5s 570us/step - loss: 0.4996 - acc: 0.7639
Epoch 3/4
8500/8500 [==============================] - 5s 566us/step - loss: 0.4179 - acc: 0.8114
Epoch 4/4
8500/8500 [==============================] - 5s 568us/step - loss: 0.3885 - acc: 0.8308
LSTM, Word Embeddings accuracy:0.7297297297297297     f1 score: 0.6862745098039216


In [0]:
#lstm bidirectional 128
def lstm(xtrain, ytrain, xvalid, yvalid, epochs = 1):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer1 =  layers.Bidirectional(layers.LSTM(128))(embedding_layer)
    dropout1 = layers.Dropout(0.5)(lstm_layer1)
    #lstm_layer2 = layers.LSTM(128)(dropout1)
    #dropout2 = layers.Dropout(0.5)(lstm_layer2)
    # Add the output Layers
    output_layer = layers.Dense(2, activation="softmax")(dropout1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=4)
    
    predictions = model.predict(xvalid)
    val_preds = predictions.argmax(axis=-1)
    accuracy = model.evaluate(xvalid, yvalid, verbose=0)
    f1score=final_score(getf1_score(np.argmax(yvalid,axis=1),val_preds),f1_weight(yvalid,val_preds))
    return accuracy, f1score
    
accuracy, f1score = lstm(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
print("LSTM, Word Embeddings accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

Epoch 1/4
8500/8500 [==============================] - 13s 2ms/step - loss: 0.5733 - acc: 0.7446
Epoch 2/4
8500/8500 [==============================] - 10s 1ms/step - loss: 0.4805 - acc: 0.7771
Epoch 3/4
8500/8500 [==============================] - 10s 1ms/step - loss: 0.4096 - acc: 0.8206
Epoch 4/4
8500/8500 [==============================] - 10s 1ms/step - loss: 0.3824 - acc: 0.8355
LSTM, Word Embeddings accuracy:0.7668918918918919     f1 score: 0.7676767676767677


In [0]:
# wiki bidir lstm 256

def lstm(xtrain, ytrain, xvalid, yvalid, epochs = 1):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer1 =  layers.Bidirectional(layers.LSTM(256))(embedding_layer)
    dropout1 = layers.Dropout(0.5)(lstm_layer1)
    #lstm_layer2 = layers.LSTM(128)(dropout1)
    #dropout2 = layers.Dropout(0.5)(lstm_layer2)
    # Add the output Layers
    output_layer = layers.Dense(2, activation="softmax")(dropout1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=4)
    
    predictions = model.predict(xvalid)
    val_preds = predictions.argmax(axis=-1)
    accuracy = model.evaluate(xvalid, yvalid, verbose=0)
#     metrcs=sklearn.metrics.precision_recall_fscore_support(np.argmax(yvalid,axis=1),val_preds)
    f1score=final_score(getf1_score(np.argmax(yvalid,axis=1),val_preds),f1_weight(yvalid,val_preds))
    return accuracy, f1score
    
accuracy, f1score = lstm(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
print("LSTM, Word Embeddings accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

In [0]:
print("LSTM, Word Embeddings accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

LSTM, Word Embeddings accuracy:0.7432432432432432     f1 score: 0.7110266159695818


In [0]:
# wiki bidir lstm 1024

def lstm(xtrain, ytrain, xvalid, yvalid, epochs = 1):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer1 =  layers.Bidirectional(layers.LSTM(1024))(embedding_layer)
    dropout1 = layers.Dropout(0.5)(lstm_layer1)
#     lstm_layer2 = layers.Bidirectional(layers.LSTM(256))(dropout1)
#     dropout2 = layers.Dropout(0.5)(lstm_layer2)
    # Add the output Layers
    output_layer = layers.Dense(2, activation="softmax")(dropout1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=3)
    
    predictions = model.predict(xvalid)
    val_preds = predictions.argmax(axis=-1)
    accuracy = model.evaluate(xvalid, yvalid, verbose=0)
#     metrcs=sklearn.metrics.precision_recall_fscore_support(np.argmax(yvalid,axis=1),val_preds)
    f1score=final_score(getf1_score(np.argmax(yvalid,axis=1),val_preds),f1_weight(yvalid,val_preds))
    return accuracy, f1score
    
accuracy, f1score = lstm(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
print("LSTM, Word Embeddings accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

**Glove Embedding**

In [0]:
!unzip '/content/drive/My Drive/glove.6B.zip' -d '/content/drive/My Drive/models/'

In [0]:
embeddings_index = dict()
f = open('/content/drive/My Drive/models/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [0]:
vocabulary_size = 20000
tokenizer = text.Tokenizer(num_words= vocabulary_size)

tokenizer.fit_on_texts(xtrain)
word_index = tokenizer.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(tokenizer.texts_to_sequences(xtrain), maxlen=50)
valid_seq_x = sequence.pad_sequences(tokenizer.texts_to_sequences(xvalid), maxlen=50)


In [0]:
embedding_matrix = np.zeros((vocabulary_size, 100))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [0]:
# convert to one_hot
train_y_onehot = keras.utils.to_categorical(ytrain, 2)
valid_y_onehot = keras.utils.to_categorical(yvalid, 2)

In [0]:
def lstm(xtrain, ytrain, xvalid, yvalid, epochs = 1):
    # Add an Input Layer
    input_layer = layers.Input((50, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(vocabulary_size, 100, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer1 = layers.LSTM(128)(embedding_layer)
    dropout1 = layers.Dropout(0.5)(lstm_layer1)
    #lstm_layer2 = layers.LSTM(128)(dropout1)
    #dropout2 = layers.Dropout(0.5)(lstm_layer2)
    # Add the output Layers
    output_layer = layers.Dense(2, activation="softmax")(dropout1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=2)
    
    predictions = model.predict(xvalid)
    val_preds = predictions.argmax(axis=-1)
    accuracy = model.evaluate(xvalid, yvalid, verbose=0)
    f1score=final_score(getf1_score(np.argmax(yvalid,axis=1),val_preds),f1_weight(yvalid,val_preds))
    return accuracy, f1score
    
accuracy, f1score = lstm(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
print("LSTM, Word Embeddings accuracy:%s     f1 score: %s"% (accuracy[1], f1score))